In [4]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.optim import lr_scheduler
import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.cuda.amp import GradScaler, autocast
# Load GoEmotions dataset
dataset = load_dataset("go_emotions")

# Check dataset structure
print(dataset)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt",
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Convert labels to multi-label format
tokenized_datasets = tokenized_datasets.map(lambda x: {"labels": [1 if i in x["labels"] else 0 for i in range(28)]})
tokenized_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)

# Prepare train, validation, and test datasets
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=28,  # GoEmotions has 28 emotion labels
    problem_type="multi_label_classification"
)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

loss_function = nn.BCEWithLogitsLoss()

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to GPU

# Training loop
scaler = GradScaler()

def train_model(model, train_loader, optimizer, loss_function, scheduler, epochs=1):
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        loop = tqdm(train_loader, leave=True)

        for batch in loop:
            # Move data to device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device).float()

            optimizer.zero_grad()

            # Mixed precision forward pass
            with autocast():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss

            # Scale the loss and backpropagate
            scaler.scale(loss).backward()

            # Step the optimizer and scaler
            scaler.step(optimizer)
            scaler.update()

            # Update metrics
            total_loss += loss.item()
            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())

        scheduler.step()
        print(f"Epoch {epoch} Loss: {total_loss / len(train_loader)}")
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Train the model
train_model(model, train_loader, optimizer, loss_function, scheduler, epochs=1)

# Save the model and tokenizer
model.save_pretrained("goemotions_bert_finetuned")
tokenizer.save_pretrained("goemotions_bert_finetuned")

# Evaluation function


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\aryan\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\aryan\AppData\Local\Temp\ipykernel_22936\1288271061.py:65: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
  0%|          | 0/1357 [00:00<?, ?it/s]C:\Users\aryan\AppData\Local\Temp\ipykernel_22936\1288271061.py:83: FutureWarning: `torch.cuda.amp.autocas

Epoch 0 Loss: 0.12171714199746718


('goemotions_bert_finetuned\\tokenizer_config.json',
 'goemotions_bert_finetuned\\special_tokens_map.json',
 'goemotions_bert_finetuned\\vocab.txt',
 'goemotions_bert_finetuned\\added_tokens.json')

In [7]:
from sklearn.metrics import accuracy_score
def evaluate_model(model, val_loader):
    model.eval()
    true_labels = []
    predictions = []
    
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device).float()
            
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.sigmoid(logits).cpu().numpy() > 0.5
            
            true_labels.extend(labels.cpu().numpy())
            predictions.extend(preds)
    
    # Compute metrics
    true_labels = np.array(true_labels)
    predictions = np.array(predictions)
    f1 = f1_score(true_labels, predictions, average="macro")
    precision = precision_score(true_labels, predictions, average="macro")
    recall = recall_score(true_labels, predictions, average="macro")
    accuracy = accuracy_score(true_labels, predictions)
    print(f"F1 Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Accuracy: {accuracy}")

# Evaluate the model
evaluate_model(model, val_loader)

# Predict function
def predict_emotions(model, tokenizer, text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to GPU
    print(inputs)
    outputs = model(**inputs)
    probs = torch.sigmoid(outputs.logits).detach().cpu().numpy()  # Move back to CPU
    return probs > 0.5  # Return multi-label predictions

# Example prediction
text = "I am so happy and excited!"
predictions = predict_emotions(model, tokenizer, text)
print(predictions)


F1 Score: 0.2964156035647725
Precision: 0.4919012771427877
Recall: 0.2488719032440476
Accuracy: 0.38997419830446
{'input_ids': tensor([[ 101, 1045, 2572, 2061, 3407, 1998, 7568,  999,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
[[False False False False False False False False False False False False
  False False False False False  True False False False False False False
  False False False False]]


c:\Users\aryan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
